<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

## Tracking Invitation Homes Quarterly Activity From the Properties V2 Endpoint

In this notebook, we will analyze Invitation Homes key metrics using [property search v2 endpoint](https://docs.parcllabs.com/reference/property_search_v2_v2_property_search_post-1), both for the current quarter (at the time of this notebook being published, halfway through Q3 2025) and historical quarters from Q1 2024 through Q2 2025. This is meant to be a rough recreation of the Parcl Labs [SFR investor report](https://app.parcllabs.com/portfolios/sfr), teaching users how to think about the events with one approach to create these key metrics. However the event history is a powerful tool and we encourage you to develop your own methodologies for metrics!

The key metrics are:
- Inventory
- Acquisitions
- Dispositions
- Net Change
- For Sale Listings
- Avg Rent
- Rent Growth

This notebook is organized into four main sections:

**Section 1: Set Up**
- Import required packages 
- Setup the Parcl Labs API key and API headers

**Section 2: Current Quarter Key Metrics**
- Inventory, Acquisitions, Dispositions, Net Change, For Sale and Avg Rent for the current quarter
- Current-state metrics use a different approach than historical metrics
- Rent growth is not included here; see Section 4 (requires historical data)
 
# <img src="outputs/INVH_Current_Key_metrics.png" alt="Invitation Homes Current Key Metrics" width="600"/>

**Section 3: Historical Point-in-Time Metrics (Low Credit Usage)**
- Acquisitions, Dispositions, Avg Rent, etc.
- Requires only 3-month event data
- Efficient for individual quarterly analysis

**Section 4: Full Event History (High Credit Usage)**  
- Quarterly inventory snapshots
- YoY rent rate analysis
- Historical on market analysis
- Requires complete ownership history
- More expensive but comprehensive

# <img src="outputs/INVH_Historical_Key_metrics.png" alt="Invitation Homes Historical Key Metrics" width="600"/>

**Note:** For production use, we recommend pulling the just the dispositions and the complete event history once each and deriving all metrics from those two datasets to avoid pulling the same events twice.

#### Why Two Different Approaches?

**Point-in-Time Metrics:** Only need time boxed events = Low credits

**Historical Inventory:** Need complete ownership history = High credits

**Pro Tip:** For the teaching purposes of this notebook, we are going to do multiple api calls for each metric in order to show how each one would be structured. For efficiency, pull the dispositions and the complete event history once each and derive all metrics from those two datasets.

**Reminders:**

- You can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

- To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`. 
- To run this notebook at scale and download data for multiple markets and endpoints, you will need to upgrade your Parcl Labs API account from free to starter to get additional credits. You can easily upgrade at any time by visiting your [Parcl Labs dashboard](https://dashboard.parcllabs.com/login), clicking "Upgrade Now" ($99, no commitment). This will unlock more credits immediately.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/housing_market_research/investor_analytics/invh_current_and_hist_key_metrics.ipynb)

### 1. Import required packages and setup the Parcl Labs API key and API headers

In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs nbformat

In [ ]:
import os
import pandas as pd
import numpy as np
from parcllabs import ParclLabsClient

In [ ]:
api_key = os.getenv('PARCL_LABS_API_KEY')
client = ParclLabsClient(api_key, num_workers=20)

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": api_key
}

### 2. Current Key Metrics

In this section we will pull the key metrics, minus Rent Growth, for the current quarter. There are parameters in the [property search v2 endpoint](https://docs.parcllabs.com/reference/property_search_v2_v2_property_search_post-1), that make pulling current state easier on the user, to show this we will pull current metrics seperately from the historical metrics. Parameters such as `current_on_market_flag` and `current_entity_owner_name` will simplify the creation of the Inventory and For Sale metrics, which we will need to calculate from the event history when creating the historical version of these metrics.

Since rent growth requires historical it is not included in this section; see Section 4 to see how to pull the Rent Growth metric.

This section will be split up into four seperate calls:
- Aquisition and Avg Rent can be done together
- Dispositions
- For Sale
- Inventory

In [ ]:
# Retrieve current quarter sales and rental events for Invitation Homes with the National Parcl ID
ih_current_q_sales_rentals_df, filter_data = client.property_v2.search.retrieve(
    parcl_ids=[5826765],
    event_names=["ALL_SOLD", "ALL_RENTALS"],
    min_event_date="2025-07-01",
    max_event_date="2025-09-30",
    owner_name=["INVITATION_HOMES"], # Filter for events where Invitation Homes was the owner
    include_property_details=False,
)

ih_current_q_sales_rentals_df

Now lets take a look at the event types and names that are available in the response, it is essential to understand the event structure in order to accurately analyze the data

In [ ]:
event_group_counts = ih_current_q_sales_rentals_df.groupby(['event_event_type', 'event_event_name']).size().reset_index(name='count')
event_group_counts = event_group_counts.sort_values(by=['event_event_type', 'event_event_name'])

print(event_group_counts)

Take not that some event names (i.e. price change) can be the same across event types. It is important when we calculate metrics to distinguish between a Rental Price Changes and a Listing Price Changes, or when rolling up acquisitions to not include Non-Arms Length transactions. 

For documentation on each `event_name` see the [property search v2 endpoint](https://docs.parcllabs.com/reference/property_search_v2_v2_property_search_post-1).

In [ ]:
# Ensure event_event_date is datetime
ih_current_q_sales_rentals_df['event_event_date'] = pd.to_datetime(ih_current_q_sales_rentals_df['event_event_date'])

# Group by event_event_date truncated to the quarter (should all be the same quarter in this df)
ih_current_q_sales_rentals_df['Quarter'] = ih_current_q_sales_rentals_df['event_event_date'].dt.to_period('Q').astype(str)
current_quarter = ih_current_q_sales_rentals_df['Quarter'].iloc[0]

# --- Acquisitions (count all SOLD and SOLD_INTER_PORTFOLIO_TRANSFER for this quarter) ---
acq_count = ih_current_q_sales_rentals_df[
    ih_current_q_sales_rentals_df['event_event_name'].isin(['SOLD', 'SOLD_INTER_PORTFOLIO_TRANSFER'])
]['parcl_property_id'].nunique()

# --- Avg Rent (mean of event_price for RENTAL events in this quarter) ---
avg_rent_value = (
    ih_current_q_sales_rentals_df[
        (ih_current_q_sales_rentals_df['event_event_type'] == 'RENTAL') &
        (ih_current_q_sales_rentals_df['event_event_name'].isin(['LISTED_RENT', 'PRICE_CHANGE']))
    ]['event_price']
    .mean()
)

# Round avg_rent_value to nearest integer if not NaN
if pd.notnull(avg_rent_value):
    avg_rent_value = int(round(avg_rent_value))
else:
    avg_rent_value = pd.NA

# Create a single-row DataFrame for the current quarter
ih_current_aquisitions_avg_rent = pd.DataFrame({
    'Quarter': [current_quarter],
    'Acquisitions': [acq_count],
    'Avg Rent': [avg_rent_value]
})

ih_current_aquisitions_avg_rent

In [ ]:
## Construct the query for current quarter IH dispositions using the National Parcl ID

ih_current_sales_dispositions, filter_data = client.property_v2.search.retrieve(
    parcl_ids=[5826765],
    event_names=["SOLD", "SOLD_INTER_PORTFOLIO_TRANSFER"],
    min_event_date="2025-07-01",
    max_event_date="2025-09-30",
    entity_seller_name=["INVITATION_HOMES"], # Filter for events where Invitation Homes was the seller
    include_property_details=False,
)

ih_current_sales_dispositions

In [ ]:
# --- Ensure event_event_date is datetime and extract quarter in one step ---
ih_current_sales_dispositions = ih_current_sales_dispositions.copy()
ih_current_sales_dispositions['event_event_date'] = pd.to_datetime(
    ih_current_sales_dispositions['event_event_date'], errors='coerce'
)
ih_current_sales_dispositions['Quarter'] = ih_current_sales_dispositions['event_event_date'].dt.to_period('Q')

# --- Group by quarter and count unique parcl_property_id dispositions ---
ih_current_dispositions = (
    ih_current_sales_dispositions
    .groupby('Quarter')['parcl_property_id']
    .nunique()
    .sort_index()
    .rename('Dispositions')
    .reset_index()
)

ih_current_dispositions

In [ ]:
# Retrieve current quarter on market properties for Invitation Homes with the National Parcl ID
ih_current_q_on_market, filter_data = client.property_v2.search.retrieve(
    parcl_ids=[5826765],
    current_on_market_flag=True, # Filter for properties that are currently on the market
    current_entity_owner_name="INVITATION_HOMES", # Filter for properties where Invitation Homes is currently the owner
    include_property_details=True, # We only need property details in this section
    include_events=False, # We will not be using events in this section
)

ih_current_q_on_market

In [ ]:
# --- Manually set the quarter to 2025Q3 and count unique parcl_property_id ---
ih_current_q_on_market = ih_current_q_on_market.copy()
ih_current_for_sale = pd.DataFrame({
    'Quarter': ['2025Q3'],
    'For Sale': [ih_current_q_on_market['parcl_property_id'].nunique()]
})

ih_current_for_sale

In [ ]:
# Retrieve current inventory for Invitation Homes with the National Parcl ID
ih_current_owner_inventory, filter_data = client.property_v2.search.retrieve(
    parcl_ids=[5826765],
    current_entity_owner_name="INVITATION_HOMES", # Filter for properties where Invitation Homes is currently the owner
    include_property_details=True, # We only need property details in this section
    include_events=False, # We will not be using events in this section
)

ih_current_owner_inventory

In [ ]:
# --- Manually set the quarter to 2025Q3 and count unique parcl_property_id ---
ih_current_owner_inventory = ih_current_owner_inventory.copy()
ih_current_inventory = pd.DataFrame({
    'Quarter': ['2025Q3'],
    'Inventory': [ih_current_owner_inventory['parcl_property_id'].nunique()]
})

ih_current_inventory

In [ ]:
# Join all current quarter dataframes into one key metrics dataframe

# Filter all dataframes to 2025Q3 and select needed columns
current_q = '2025Q3'

# Create a dictionary of the dataframes with their key columns
dfs_dict = {
    'Inventory': ih_current_inventory[ih_current_inventory['Quarter'] == current_q][['Quarter', 'Inventory']],
    'Acquisitions': ih_current_aquisitions_avg_rent[ih_current_aquisitions_avg_rent['Quarter'] == current_q][['Quarter', 'Acquisitions']],
    'Dispositions': ih_current_dispositions[ih_current_dispositions['Quarter'] == current_q][['Quarter', 'Dispositions']],
    'For Sale': ih_current_for_sale[ih_current_for_sale['Quarter'] == current_q][['Quarter', 'For Sale']],
    'Avg Rent': ih_current_aquisitions_avg_rent[ih_current_aquisitions_avg_rent['Quarter'] == current_q][['Quarter', 'Avg Rent']]
}

# Convert all Quarter columns to string to ensure compatibility
for name, df in dfs_dict.items():
    dfs_dict[name]['Quarter'] = dfs_dict[name]['Quarter'].astype(str)

# Merge all dataframes on Quarter
ih_current_key_metrics = None
for name, df in dfs_dict.items():
    if ih_current_key_metrics is None:
        ih_current_key_metrics = df
    else:
        ih_current_key_metrics = pd.merge(ih_current_key_metrics, df, on='Quarter', how='outer')

# Calculate Net Change
ih_current_key_metrics['Net Change'] = ih_current_key_metrics['Acquisitions'] - ih_current_key_metrics['Dispositions']

# Reorder columns to match desired output
final_columns = ['Quarter', 'Inventory', 'Acquisitions', 'Dispositions', 'Net Change', 'For Sale', 'Avg Rent']
ih_current_key_metrics = ih_current_key_metrics[final_columns]

ih_current_key_metrics

We have our final dataframe with all Invitation Homes ket metrics for the current quarter! Let's move on to the historical analysis.

### 3. Leverage the V2 Prop Endpoint for Historical Point in Time Metrics (Aquisitions, Dispositions and Rents)

All of these metrics will look at data that is grouped quarterly, we can do this most efficiently by pulling all Invitation Homes activity between 1/1/2024 and 06/30/2025 in one query (~37,000 credits) and then analyze the resulting dataframe for all metrics where they were the owner.

For dispositions where Invitation Homes was the seller, we will have to pull those events seperately.

For both calls we will use the [property search v2 endpoint](https://docs.parcllabs.com/reference/property_search_v2_v2_property_search_post-1), see the documentation for information on parameters the response.

In [ ]:
## Construct the query for all IH Sale and Rental events from  1/1/2024 to 6/30/2025 using the National Parcl ID

ih_2024_2025_sales_rentals_df, filter_data = client.property_v2.search.retrieve(
    parcl_ids=[5826765],
    event_names=["ALL_SOLD", "ALL_RENTALS"], # Filter for only sales and rental events
    min_event_date="2024-01-01", # Filter for events from 1/1/2024 to 6/30/2025
    max_event_date="2025-06-30",
    owner_name=["INVITATION_HOMES"], # Filter for events where Invitation Homes was the owner
    include_property_details=False, # We will not be using property details in this section
)

ih_2024_2025_sales_rentals_df

In [ ]:
# Ensure event_event_date is datetime and create a 'quarter' column
ih_2024_2025_sales_rentals_df['event_event_date'] = pd.to_datetime(ih_2024_2025_sales_rentals_df['event_event_date'])
ih_2024_2025_sales_rentals_df['Quarter'] = ih_2024_2025_sales_rentals_df['event_event_date'].dt.to_period('Q')

# --- Acquisitions (count all SOLD and SOLD_INTER_PORTFOLIO_TRANSFER per quarter) ---
acq = (
    ih_2024_2025_sales_rentals_df[ih_2024_2025_sales_rentals_df['event_event_name'].isin(['SOLD', 'SOLD_INTER_PORTFOLIO_TRANSFER'])]
    .groupby('Quarter')
    .size()
    .reset_index(name='Acquisitions')
    .sort_values('Quarter')
    .reset_index(drop=True)
)

# --- Avg Rent (mean of event_price for all RENTAL listing events per quarter) ---
avg_rent = (
    ih_2024_2025_sales_rentals_df[
        (ih_2024_2025_sales_rentals_df['event_event_type'] == 'RENTAL') &
        (ih_2024_2025_sales_rentals_df['event_event_name'].isin(['LISTED_RENT', 'PRICE_CHANGE']))
    ]
    .groupby('Quarter')['event_price']
    .mean()
    .round(0)
    .astype('Int64')
    .reset_index(name='Avg Rent')
    .sort_values('Quarter')
    .reset_index(drop=True)
)

# Combine into a single DataFrame
ih_historical_aquisitions_avg_rent = pd.merge(acq, avg_rent, on='Quarter', how='outer').sort_values('Quarter').reset_index(drop=True)

ih_historical_aquisitions_avg_rent

Now that we have our point in time Acquisitons and Avg Rent Rates, we can move on to Dispositions and Net Change metrics. We will construct a similar query as our first one, changing the event names (we only need sales) and specifically querying on `entity_seller_name`. 

Note that when looking at a sale transactions, the `entity_seller_name` will be the seller and the `entity_owner_name` is the buyer. `entity_seller_name` will only be populated on SALE event types, while `entity_owner_name` can be on any event type. 

In [ ]:
## Construct the query for all IH dispositions in between 1/1/2024 and 6/30/2025 using the National Parcl ID

ih_2024_2025_dispositions, filter_data = client.property_v2.search.retrieve(
    parcl_ids=[5826765],
    event_names=["SOLD", "SOLD_INTER_PORTFOLIO_TRANSFER"], # Filter for only sales and inter portfolio transfer events
    min_event_date="2024-01-01", # Filter for events from 1/1/2024 to 6/30/2025
    max_event_date="2025-06-30",
    entity_seller_name=["INVITATION_HOMES"], # Filter for events where Invitation Homes was the seller
    include_property_details=False, # We will not be using property details in this section
)

ih_2024_2025_dispositions

In [ ]:
# --- Format event_event_date and extract quarter ---
ih_2024_2025_dispositions['event_event_date'] = pd.to_datetime(
    ih_2024_2025_dispositions['event_event_date']
)
ih_2024_2025_dispositions['Quarter'] = (
    ih_2024_2025_dispositions['event_event_date'].dt.to_period('Q')
)

# --- Group by quarter and count dispositions ---
ih_historical_dispositions = (
    ih_2024_2025_dispositions
    .groupby('Quarter')
    .size()
    .reset_index(name='Dispositions')
    .sort_values('Quarter')
    .reset_index(drop=True)
)

ih_historical_dispositions

Now that we have our quarterly point in time metrics, we can move on to inventory and for sale listings, for which we will need the full event history.

### 4. Use the V2 Prop Endpoint for Historical Metrics that Require the Full Event History

All of these metrics will look at data that is grouped quarterly, but we will need the entire event history of the home in order to calculate them. For example in order to know who owned a home at the end of Q2 2024, you will need the latest sale event for that home, regardless of when that sale occurred.

Notice how we use the `include_full_event_history` parameter to achieve this - in order for the full event history to be returned, the parameter `include_events` must also be true. This parameter defaults to true but we will be explicit in this call. This call is much more expensive than the previous calls (100,000+ credits)

As usual we will use the [property search v2 endpoint](https://docs.parcllabs.com/reference/property_search_v2_v2_property_search_post-1), see the documentation for information on parameters the response.

In [ ]:
## Construct the query for all events on any home Invitation Homes has ever owned
ih_full_history_df, filter_data = client.property_v2.search.retrieve(
    parcl_ids=[5826765],
    owner_name=["INVITATION_HOMES"], # Filter for events where Invitation Homes was the owner
    include_property_details=False, # We will not be using property details in this section
    include_full_event_history=True, # We will be using the full event history
    include_events=True, # We will be using events in this section
)

ih_full_history_df

Now that we have the full history for all the homes IH has ever owned, we can drill down into historical inventory and on market metrics. We will calculate inventory by taking the latest sale event for each home Invitation Homes ever owned at each quarter end, and if that sale is attributed to INVH then we can add it as part of their inventory.

In [ ]:
# Define the quarter labels and their corresponding end dates for Q1 2024 to Q2 2025
quarter_labels = [
    '2024Q1',
    '2024Q2',
    '2024Q3',
    '2024Q4',
    '2025Q1',
    '2025Q2'
]
quarter_ends = [
    '2024-03-31',
    '2024-06-30',
    '2024-09-30',
    '2024-12-31',
    '2025-03-31',
    '2025-06-30'
]

# Ensure event_event_type and event_event_date columns exist and are in correct format
df = ih_full_history_df.copy()
df = df[df['event_event_type'] == 'SALE'].copy()
df['event_event_date'] = pd.to_datetime(df['event_event_date'])

inventory_by_quarter = []

for q_label, q_end in zip(quarter_labels, quarter_ends):
    q_end_date = pd.to_datetime(q_end)
    # For each property, get the latest SALE event on or before the quarter end
    df_q = df[df['event_event_date'] <= q_end_date]
    # Get the max event date per property
    idx = df_q.groupby('parcl_property_id')['event_event_date'].idxmax()
    latest_sales = df_q.loc[idx]
    # Count homes where owner is Invitation Homes
    owned_count = (latest_sales['event_entity_owner_name'].str.upper() == 'INVITATION_HOMES').sum()
    inventory_by_quarter.append({'Quarter': q_label, 'Inventory': owned_count})

ih_historical_inventory = pd.DataFrame(inventory_by_quarter)
ih_historical_inventory

It's currently not possible to recreate historical on market listings for an owner in the same manner that Parcl Labs creates it's on market metric in the [SFR investor report](https://app.parcllabs.com/portfolios/sfr). This is because Parcl Labs maintains historical on market snaphots that are not currently released in the API at this time. 

Below is one example of how one might try and create a historical on market inventory at quarter end, by seeing what homes were listed in each quarter and not delisted by quarter end. These are not meant to be official numbers, but rather give you an idea of how to work with the data and create your own history for this metric, we encourage you to develop your own methodology.

In [ ]:
# Calculate homes listed for sale at each quarter end
df_all = ih_full_history_df.copy()
df_all['event_event_date'] = pd.to_datetime(df_all['event_event_date'])

listed_for_sale_by_quarter = []

for q_label, q_end in zip(quarter_labels, quarter_ends):
    q_end_date = pd.to_datetime(q_end)
    # 1. Find all listings within the quarter up to quarter end
    listings = df_all[
        (df_all['event_event_type'] == 'LISTING') &
        (df_all['event_event_name'].isin(['LISTED_SALE'])) &
        (df_all['event_event_date'] > (q_end_date - pd.offsets.QuarterEnd(startingMonth=3))) & (df_all['event_event_date'] <= q_end_date) &
        (df_all['event_entity_owner_name'].str.upper() == 'INVITATION_HOMES')
    ].copy()

    # 2. For each listed property, check if there is a removal event before or at quarter end, after the listing date
    listed_ids = listings['parcl_property_id'].unique()
    qualifying_ids = set()

    for prop_id in listed_ids:
        prop_events = df_all[
            (df_all['parcl_property_id'] == prop_id) &
            (df_all['event_event_date'] <= q_end_date)
        ].sort_values('event_event_date')

        # Find the latest qualifying listing event before or at quarter end
        listing_events = prop_events[
            (prop_events['event_event_type'] == 'LISTING') &
            (prop_events['event_event_name'].isin(['LISTED_SALE', 'PRICE_CHANGE', 'RELISTED']))
        ]
        if listing_events.empty:
            continue
        latest_listing = listing_events.iloc[-1]
        listing_date = latest_listing['event_event_date']

        # Check for any removal event after the listing date and before or at quarter end
        removal_events = prop_events[
            (
                (prop_events['event_event_type'] == 'SALE') |
                (
                    (prop_events['event_event_type'] == 'LISTING') &
                    (prop_events['event_event_name'] == 'LISTING REMOVED')
                )
            ) &
            (prop_events['event_event_date'] > listing_date)
        ]
        # If no removal event, count as listed for sale at quarter end
        if removal_events.empty:
            qualifying_ids.add(prop_id)

    listed_for_sale_by_quarter.append({'Quarter': q_label, 'For Sale': len(qualifying_ids)})

ih_historical_listed_for_sale = pd.DataFrame(listed_for_sale_by_quarter)
print(ih_historical_listed_for_sale)

For historical rent growth metrics, Parcl Labs takes homes that were listed for rent a year prior by the same entity and compares those rents to the quarter that the metric is being calculated for. We then determine the percent changes in leases for the same home year over year, and take the average changes to get one rent growth metric. Due to the data stewardship rules implemented in the api data (cleanliness, lags etc.) these numbers will slightly differ from the app's SFR dashboard. 

This metric is meant to help determine the change in revenue over time for rentals of an entity, but once again we encourage users to develop their own methodologies to track rent dynamics over time.

In [ ]:
# Calculate quarterly percent rental growth to match dbt methodology

# Define reporting quarters
quarters = pd.period_range('2024Q1', '2025Q2', freq='Q')

allowed_rental_event_names = {'LISTED_RENT', 'PRICE_CHANGE', 'RELISTED'}

core = ih_full_history_df.copy()
core['event_event_date'] = pd.to_datetime(core['event_event_date'])

# Sales by INVITATION_HOMES (ownership universe by quarter)
sales_df = core[
    (core['event_event_type'] == 'SALE') &
    (core['event_entity_owner_name'] == 'INVITATION_HOMES')
][['parcl_property_id', 'event_event_date', 'event_true_sale_index', 'event_transfer_index', 'event_entity_owner_name']].copy()

# Rental events (candidate set)
rentals_df = core[
    (core['event_event_type'] == 'RENTAL') &
    (core['event_event_name'].isin(allowed_rental_event_names))
][['parcl_property_id', 'event_true_sale_index', 'event_event_name', 'event_event_date', 'event_price']].copy()
rentals_df['event_quarter'] = rentals_df['event_event_date'].dt.to_period('Q')

# Precompute previous rental event of the same type roughly one year earlier (11–13 months)
rentals_df = rentals_df.sort_values(['parcl_property_id', 'event_true_sale_index', 'event_event_name', 'event_event_date'])
rentals_df['prev_event_date'] = rentals_df.groupby(
    ['parcl_property_id', 'event_true_sale_index', 'event_event_name']
)['event_event_date'].shift(1)
rentals_df['prev_event_price'] = rentals_df.groupby(
    ['parcl_property_id', 'event_true_sale_index', 'event_event_name']
)['event_price'].shift(1)

months_diff = (
    (rentals_df['event_event_date'].dt.year - rentals_df['prev_event_date'].dt.year) * 12
    + (rentals_df['event_event_date'].dt.month - rentals_df['prev_event_date'].dt.month)
)
rentals_df['prev_is_yoyish'] = months_diff.between(11, 13, inclusive='both')
rentals_df.loc[~rentals_df['prev_is_yoyish'], ['prev_event_date', 'prev_event_price']] = np.nan

results = []
for q in quarters:
    # Latest sale per property up to this quarter (includes sales within the quarter)
    q_end = q.end_time
    sales_q = sales_df[sales_df['event_event_date'] <= q_end].copy()
    if 'event_transfer_index' in sales_q.columns:
        sales_q = sales_q.sort_values(['parcl_property_id', 'event_transfer_index'], ascending=[True, False])
    else:
        sales_q = sales_q.sort_values(['parcl_property_id', 'event_event_date'], ascending=[True, False])
    latest_sale_per_property = sales_q.drop_duplicates(subset=['parcl_property_id'], keep='first')[
        ['parcl_property_id', 'event_true_sale_index']
    ]

    # Rentals eligible up to this quarter (DATE_TRUNC('quarter', rental_date) <= quarter_start)
    rentals_q = rentals_df[rentals_df['event_quarter'] <= q].copy()

    # Restrict rentals to current ownership epoch via true_sale_index
    rentals_owned = rentals_q.merge(
        latest_sale_per_property,
        on=['parcl_property_id', 'event_true_sale_index'],
        how='inner'
    )

    if rentals_owned.empty:
        results.append({'Quarter': str(q), 'Rent Growth': None})
        continue

    # Latest rental per property up to the quarter (tie-break to match SQL: date desc, price asc, name asc)
    rentals_owned = rentals_owned.sort_values(
        ['parcl_property_id', 'event_event_date', 'event_price', 'event_event_name'],
        ascending=[True, False, True, True]
    )
    latest_rental_per_property = rentals_owned.drop_duplicates(subset=['parcl_property_id'], keep='first')

    valid = latest_rental_per_property['prev_event_price'].notna() & (latest_rental_per_property['prev_event_price'] > 0)
    if not valid.any():
        results.append({'Quarter': str(q), 'Rent Growth': None})
        continue

    avg_delta = (latest_rental_per_property.loc[valid, 'event_price'] - latest_rental_per_property.loc[valid, 'prev_event_price']).mean()
    avg_prev = latest_rental_per_property.loc[valid, 'prev_event_price'].mean()
    percent_growth = round((avg_delta / avg_prev) * 100, 2)

    results.append({'Quarter': str(q), 'Rent Growth': percent_growth})

ih_historical_rent_growth = pd.DataFrame(results)
print(ih_historical_rent_growth)

In [ ]:
# Join all historical dataframes into one key metrics dataframe

# Create a dictionary of the historical dataframes with their key columns
historical_dfs_dict = {
    'Inventory': ih_historical_inventory[['Quarter', 'Inventory']].copy(),
    'Acquisitions': ih_historical_aquisitions_avg_rent[['Quarter', 'Acquisitions']].copy(),
    'Dispositions': ih_historical_dispositions[['Quarter', 'Dispositions']].copy(),
    'For Sale': ih_historical_listed_for_sale[['Quarter', 'For Sale']].copy(),
    'Avg Rent': ih_historical_aquisitions_avg_rent[['Quarter', 'Avg Rent']].copy(),
    'Rent Growth': ih_historical_rent_growth[['Quarter', 'Rent Growth']].copy()
}

# Convert all Quarter columns to string to ensure compatibility
for name, df in historical_dfs_dict.items():
    historical_dfs_dict[name]['Quarter'] = historical_dfs_dict[name]['Quarter'].astype(str)

# Start with the first dataframe
ih_historical_key_metrics = historical_dfs_dict['Inventory'].copy()

# Merge with each subsequent dataframe
for name, df in historical_dfs_dict.items():
    if name != 'Inventory':  # Skip the first one since we already have it
        ih_historical_key_metrics = pd.merge(ih_historical_key_metrics, df, on='Quarter', how='outer')

# Calculate Net Change
ih_historical_key_metrics['Net Change'] = ih_historical_key_metrics['Acquisitions'] - ih_historical_key_metrics['Dispositions']

# Reorder columns to match current df structure with Rent Growth last
final_columns = ['Quarter', 'Inventory', 'Acquisitions', 'Dispositions', 'Net Change', 'For Sale', 'Avg Rent', 'Rent Growth']
ih_historical_key_metrics = ih_historical_key_metrics[final_columns]

ih_historical_key_metrics

Finally we have a historical analysis of the key Invitation Homes metrics. Remember, this cookbook is meant to teach you how to use the event histories to analyze entities, but we encourage you to leverage the powerful V2 endpoints to develop your own methods!